In [ ]:
# standard imports
import numpy as np
import math
import os
import csv
from tqdm import tqdm
import meshio
import matplotlib.pyplot as plt

In [ ]:
# custom class imports
from node import node
from cell import cell
from mesh import exp_mesh
from derivatives import finite_differences,polynomial_derivatives
from derivatives import bilinear_derivatives
from isentropic_navier_stokes import Q,L,inner_product

# utility functions
from utilities_modified import *

# Setting Custom Problem

In [ ]:
# read su2 mesh file
meshio_mesh = meshio.read("./lam_cyl_comp_air/mesh_cylinder_quad.su2",file_format="su2")
# convert mesh into custom class exp_mesh
mesh = exp_mesh(meshio_mesh,[0.5,0])

# Loading Filedata

In [ ]:
# reading file data
data_dir="./lam_cyl_comp_air/mach 0.01/"
data_dict = read_csv_data(data_dir,delay=0,subsampling=2,max_size=180)

In [ ]:
x = data_dict["x"][:,0] - 0.5
y = data_dict["y"][:,0]
u = np.divide(data_dict['Momentum_x'],data_dict['Density'])
v = np.divide(data_dict['Momentum_y'],data_dict['Density'])
a = data_dict['Mach']

In [ ]:
# Coordinates of Mesh Cell Centers
x_cell = np.empty(mesh.N)
y_cell = np.empty(mesh.N)

for cel in mesh.cells:
    i = cel.index
    [x_cell[i],y_cell[i]] = cel.center

In [ ]:
q = np.vstack([u,v,a])
q_avg = np.mean(q,1)
q_fluc = q - np.repeat(np.expand_dims(q_avg,1),q.shape[1],axis=1)

# Geometric Constants

In [ ]:
# DEFINING CONSTANTS
n = mesh.n      # number of nodes
N = mesh.N      # number of cells
T = q.shape[1]  # number of snapshots
alpha = 1/1.4       # inner product coefficient
method="fd"     # method for derivative computation
gamma = 1.4     # isentropic coefficient -> material property

In [ ]:
# Coordinates of Mesh Nodes
x = data_dict["x"][:,0] - 0.5
y = data_dict["y"][:,0]

# Coordinates of Mesh Cell Centers
x_cell = np.empty(N)
y_cell = np.empty(N)

for cel in mesh.cells:
    i = cel.index
    [x_cell[i],y_cell[i]] = cel.center

# POD Decomposition

In [ ]:
# computing correlation matrix with isenctropic inner product
C = np.empty((T,T))
for i in tqdm(range(T)):
    for j in range(i,T):
        C[i,j] = (1/T) * inner_product(mesh,q_fluc[:,i],q_fluc[:,j],alpha)
        C[j,i] = C[i,j]

In [ ]:
def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)

In [ ]:
S, V =  np.linalg.eigh(C)
S = np.flip(S,0)
V = np.flip(V,1)
epsilon = 0.97 # [0,1]
acc = 0
num = 0
while acc < epsilon:
    num+=1
    acc = sum(S[:num])/sum(S)    
print("Number of eigenvectors necessary for thresholded reconstruction: \t" + str(num))
if num % 2 == 1:
    num+=1

num = 20

In [ ]:
# construct spatial POD Modes - not normalized
pod_modes = np.zeros((3*n,T))
for i in range(len(S)):
    pod_modes += np.outer(q_fluc[:,i],V[i,:])

# normalizing POD Modes
for i in range(len(S)):
    pod_modes[:,i]/=np.sqrt(inner_product(mesh,pod_modes[:,i],pod_modes[:,i],alpha))

In [ ]:
data = pod_modes[:n,0]
plot_cylinder_data(x,y,pod_modes[0*n:n,6],zoom=True)

In [ ]:
# incorrect because POD modes are not orthonormal in euclidean space but in problem space
coeffs = np.matmul(np.linalg.pinv(pod_modes),q_fluc)

In [ ]:
test = np.empty(180)
for i in range(180):
    test[i] = inner_product(mesh,pod_modes[:,0],q_fluc[:,i],alpha)

In [ ]:
plot_activations(coeffs,num)

# Reconstruction of data

In [ ]:
# reconstruction 
rec = np.zeros((3*n,T))
rec += np.repeat(np.expand_dims(q_avg,1),T,1)
rec += np.matmul(pod_modes[:,:num],coeffs[:num,:])

## Galerkin System

In [ ]:
method = "fd"
output = "node"
dim = n

# Computation of Galerkin system parameters
Qavg = Q(mesh,q_avg,q_avg,method=method,output=output)
Lavg = L(mesh,q_avg,method=method,output=output)

# initialie arrays for parameters
b1 = np.empty(num)
b2 = np.empty(num)
L1 = np.empty((num,num))
L2 = np.empty((num,num))
Q_ = [np.empty((num,num,)) for x in range(num)]

# pod basis for computation
if output=="cell":
    modes = np.empty((3*dim,num))
    for i in range(num):
        modes[:N,i] = mesh.compute_cell_values_from_node_data(pod_modes[:n,i])
        modes[N:2*N,i] = mesh.compute_cell_values_from_node_data(pod_modes[n:2*n,i])
        modes[2*N:3*N,i] = mesh.compute_cell_values_from_node_data(pod_modes[2*n:3*n,i])
else:
    modes = pod_modes


# compute L and Q operators for projection
Q_tmp1 = np.empty((num,3*dim))
Q_tmp2 = np.empty((num,3*dim))
Q_tmp3 = np.empty((num,num,3*dim))
L_tmp = np.empty((num,3*dim))

for i in tqdm(range(num)):
    Q_tmp1[i] = Q(mesh,q_avg,pod_modes[:,i],method=method,output=output)
    Q_tmp2[i] = Q(mesh,pod_modes[:,i],q_avg,method=method,output=output)
    L_tmp[i] = L(mesh,pod_modes[:,i],method=method,output=output)
    for j in range(num):
        Q_tmp3[i,j] = Q(mesh,pod_modes[:,i],pod_modes[:,j],method=method,output=output)

# compute ODE coefficients
for k in tqdm(range(num)):
    b1[k] = inner_product(mesh,Lavg,pod_modes[:,k],alpha)
    b2[k] = inner_product(mesh,Qavg,pod_modes[:,k],alpha)
    for i in range(num):
        L1[k,i] = inner_product(mesh,L_tmp[i],pod_modes[:,k],alpha)
        L2[k,i] = inner_product(mesh,np.add(Q_tmp1[i],Q_tmp2[i]),pod_modes[:,k],alpha)
        for j in range(num):
            Q_[k][i,j] = inner_product(mesh,Q_tmp3[i,j],pod_modes[:,k],alpha)

# Solving ODE system

In [ ]:
# initial conditions for the activations are obtained as activations on the initial timeframe of the data matrix
a0 = coeffs[:num,0] #initial conditions
dt = 0.001
t0 = 0
tmax = 10
sampling_span = np.linspace(t0,tmax-dt,num=int(tmax/dt))
# static viscosity
nu =  1.516e-5

# defining ode function for node based coefficients
def galerkin_system(t,a):
    # set global variables references
    global Q_, L1, L2, b1, b2, nu
    
    a_dot = np.empty_like(a)
    
    for k in range(a_dot.shape[0]):
        a_dot[k] = nu * b1[k] + b2[k] + np.inner((nu*L1[k,:]+L2[k,:]),a) + np.matmul(np.matmul(np.expand_dims(a,1).T,Q_[k]),np.expand_dims(a,1))
    return a_dot

In [ ]:
from scipy.integrate import solve_ivp
sol = solve_ivp(galerkin_system,(t0,tmax),a0,method='LSODA',t_eval=sampling_span)
sol

In [ ]:
def reconstruct_flow(activations,U,avg,num,rang=(0,100)):
    low,high = rang
    flow = np.repeat(np.expand_dims(avg,1),high-low,1)
    for i in range(num):
         flow += np.outer(U[:,i],activations[i,low:high])
    return flow

In [ ]:
# Galerkin model based on discrete reconstruction as q = \bar{q}+\sum_i^n a_i(t)*\phi_i(x)
d = int(num/2)
fig,ax = plt.subplots(d,2,figsize=(15,10))
fig.tight_layout(pad=3.0)

t = sampling_span

# add plots over full time domain
for i in range(num):
    if num <= 2:
        ax[int(i%2)].plot(t,sol.y[i,:len(t)])
        ax[int(i%2)].title.set_text("Activation for Eigenflow " + str(i))
    else:
        ax[int(i/2)][int(i%2)].plot(t,sol.y[i,:len(t)])
        ax[int(i/2)][int(i%2)].title.set_text("Activation for Eigenflow " + str(i))